In [2]:
import Chess
using NBInclude

In [3]:
@nbinclude "Values.ipynb"
@nbinclude "Chess.ipynb"

getNextMove (generic function with 4 methods)

In [4]:
function alphabetaMax(extboard::ExtendedBoard, depth::Int64, α::Int32, β::Int32)::Int32
    legalMoves = Chess.moves(extboard.board)
    if length(legalMoves) == 0 || extboard.repetionRuleDraw
        return evaluateTerminalPositionScore(extboard.board) - depth
    elseif depth == 0
        return extboard.score
    end
    for move ∈ legalMoves
        undo = domove!(extboard, move)
        value = alphabetaMin(extboard, depth - 1, α, β)
        undomove!(extboard, undo)
        if value > α
            if value >= β
                return value
            end
            α = value
        end
    end
    Chess.recycle!(legalMoves)
    return α
end

alphabetaMax (generic function with 1 method)

In [11]:
function alphabetaMin(extboard::ExtendedBoard, depth::Int64, α::Int32, β::Int32)::Int32
    legalMoves = Chess.moves(extboard.board)
    if length(legalMoves) == 0 || extboard.repetionRuleDraw
        return evaluateTerminalPositionScore(extboard.board) - depth
    elseif depth == 0
        return extboard.score
    end
    for move ∈ legalMoves
        undo = domove!(extboard, move)
        value = alphabetaMax(extboard, depth - 1, α, β)
        undomove!(extboard, undo)
        if value < β
            if value <= α
                return value
            end
            β = value
        end
    end
    Chess.recycle!(legalMoves)
    return β
end

alphabetaMin (generic function with 1 method)

In [8]:
function alphabetaOne(extboard::ExtendedBoard, depth::Int64)::Tuple{Int32, Chess.Move}
    legalMoves = Chess.moves(extboard.board)
    sideIsWhite = Chess.sidetomove(extboard.board) == Chess.WHITE
    α = typemin(Int32)
    β = typemax(Int32)
    bestMove = legalMoves[1]
    for move ∈ legalMoves
        undo = domove!(extboard, move)
        value = (sideIsWhite ? alphabetaMin : alphabetaMax)(extboard, depth - 1, α, β)
        if sideIsWhite
            if value > α
                bestMove = move
                println("$(α) $(β) $(bestMove)")
                if value >= β
                    return value
                end
                α = value
            end
        else
            if value < β
                bestMove = move
                println("$(α) $(β) $(bestMove)")
                if value <= α
                    return value
                end
                β = value
            end
        end
        undomove!(extboard, undo)
    end
    return (sideIsWhite ? α : β), bestMove
end

minimaxOne (generic function with 1 method)

In [13]:
function alphabetaAll(extboard::ExtendedBoard, depth::Int64)::Tuple{Int32, Vector{Chess.Move}}
    sideIsWhite = Chess.sidetomove(extboard.board) == Chess.WHITE
    scoredMoves = Dict{Int32, Vector{Chess.Move}}()
    pruneFunction = sideIsWhite ? alphabetaMax : alphabetaMin
    bestScore = pruneFunction(extboard, depth, typemin(Int32), typemax(Int32))
    pruneFunction = sideIsWhite ? alphabetaMin : alphabetaMax
    for move ∈ Chess.moves(extboard.board)
        undo = domove!(extboard, move)
        score = pruneFunction(extboard, depth - 1, typemin(Int32), typemax(Int32))
        undomove!(extboard, undo)
        movesWithSameScore = get(scoredMoves, score, Chess.Move[])
        push!(movesWithSameScore, move)
        scoredMoves[score] = movesWithSameScore
    end
    bestScore = sideIsWhite ? typemin(Int32) : typemax(Int32)
    for score ∈ keys(scoredMoves)
        bestScore = (sideIsWhite ? max : min)(score, bestScore)
    end
    return bestScore, scoredMoves[bestScore]
end

alphabetaAll (generic function with 1 method)

In [15]:
@time alphabetaAll(ExtendedBoard(Chess.startboard()), 1)

  0.000105 seconds (145 allocations: 87.742 KiB)


(50, Chess.Move[Move(b1c3), Move(g1f3)])